# Step 03
# Predict segmentations

In [ ]:
%matplotlib inline

In [ ]:
import os
import os.path

import matplotlib.pyplot as plt
import numpy as np

import skimage.io
import skimage.morphology

import tensorflow as tf
import keras

import utils.metrics
import utils.model_builder
import math

# Configuration

In [ ]:
from config import config_vars
# Partition of the data to make predictions (test or validation)
partition = "test"
experiment_name = '12'
config_vars = utils.dirtools.setup_experiment(config_vars, experiment_name)
data_partitions = utils.dirtools.read_data_partitions(config_vars)
config_vars
print(data_partitions['test'])

In [ ]:
# Configuration to run on GPU
configuration = tf.ConfigProto()
configuration.gpu_options.allow_growth = True
configuration.gpu_options.visible_device_list = "0"
session = tf.Session(config = configuration)
keras.backend.set_session(session)

In [ ]:
image_names = [os.path.join(config_vars["normalized_images_dir"], f) for f in data_partitions[partition]]
imagebuffer = skimage.io.imread_collection(image_names)
images = imagebuffer.concatenate()
dim1 = images.shape[1]
dim2 = images.shape[2]
print(dim1, dim2)
print(images.shape)
images = images.reshape((-1, dim1, dim2, 1))
# preprocess (assuming images are encoded as 8-bits in the preprocessing step)
images = images / 255
print(images.shape)
# build model and load weights

model = utils.model_builder.get_model_3_class(dim1, dim2)
##load the weights of the saved model in out folder.
model.load_weights(config_vars["model_file"])
print("hi")
# Normal prediction time
predictions = model.predict(images, batch_size=1)
model.summary()

# Run segmentation of images 

In [ ]:
for i in range(len(images)):
    filename = imagebuffer.files[i]
    filename = os.path.basename(filename)    
    probmap = predictions[i].squeeze()
    #plt.imshow(probmap)
    #plt.show()
    skimage.io.imsave(config_vars["probmap_out_dir"] + filename, probmap) 
    pred = utils.metrics.probmap_to_pred(probmap, config_vars["boundary_boost_factor"])
    #plt.imshow(pred)
    #plt.show()
    label = utils.metrics.pred_to_label(pred, config_vars["cell_min_size"])
    #plt.imshow(label)
    #plt.show()
    skimage.io.imsave# Transform predictions to label matrices
    (config_vars["labels_out_dir"] + filename, label)

In [ ]:
print("ciao")